In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

ens='cA211.530.24'
lat=yu.LatticeEnsemble(ens)

yu.flag_fast=True

In [ ]:
path='/project/s1174/lyan/code/projectData/NST_a/cA211.530.24/data_merge/NST_a.h5_main'
data=yu.load(path)
Ncfg=yu.deepKey(data['2pt'],2).shape[0]
print('Ncfg='+str(Ncfg))

In [ ]:
flags={
    'cc2pt':False,
    'cc3pt':False,
    # 'l1l2':False,
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    t=data['2pt'][opa+'_'+opb] if opa+'_'+opb in data['2pt'] else {}
    res=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if yu.getNpars(opa,opb)==(2,1):
        res+=np.conj(get2pt(opb,opa,set(diags)-{'N-pi0i',}))
    if opb+'_'+opa in data['2pt'].keys(): # contibution for c.c of T-pi0i
        t=data['2pt'][opb+'_'+opa]; diag='T-pi0f'
        res+= np.conj(t[diag]) if diag in t.keys() and diag in diags else 0
    return res
def get2ptMat(ops,diags=yu.diags_all):
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    t=data['3pt'][opa+'_'+opb][insert] if opa+'_'+opb in data['3pt'] else {}
    res=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if yu.getNpars(opa,opb)==(2,1):
        res+=np.conj(get3pt(opb,opa,insert,set(diags)-{'N-jPi','N-j-pi0i'})[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all,NpiNpi=False):
    skipNpiNpi=lambda opa,opb: yu.getNpars(opa,opb)==(2,2) and not NpiNpi
    t=np.transpose([[get3pt(opa,opb,insert,diags) if not skipNpiNpi(opa,opb) else np.zeros([Ncfg,int(insert.split('_')[-1])+1])  for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags,NpiNpi)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

In [ ]:
tfs=[10,12,14]
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,:,0]/t[:,None]
    # return t[10]
    return np.mean(t[10:14+1,:],axis=0)

def runFF(ptofas,ptofbs,FF,tRefs=[1],label='',ylim1=None):
    
    mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
    t_Fpi=92.9/lat.aInv
    t_aQ2=(qk**2-(EN-mN)**2)
    t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
    t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
    t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P
    
    t_A2pNN_0=mN/t_Fpi
    t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
    t_52pNN_1=(lat.amu/lat.ZP)*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
    t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1
    
    t_GAzz_00=(-1)*(2*mN*(mN+mN))/(2j*((mN+mN)**2)) * lat.ZA; t_GAzzb_00=t_GAzz_00 * t_A2pNN_0
    t_GAtz_11=(-1)*(2*EN*(EN+mN))/(-4*(EN+mN)*qk) * lat.ZA; t_GAtzb_11=t_GAtz_11 * t_A2pNN_0
    t_GAzz_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2+qk**2)) * lat.ZA; t_GAzzb_11=t_GAzz_11 * t_A2pNN_0
    t_GAxx_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2)) * lat.ZA; t_GAxxb_11=t_GAxx_11 * t_A2pNN_0
    
    t_GA_01=(-1)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * lat.ZA
    t_G5_01=(-1*np.sqrt(EN*(EN+mN)/2))/qk * lat.ZS; t_G5b_01=t_G5_01 * t_52pNN_1
    t_GP_01_tz=(-1)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * lat.ZA; t_GP_01_tz_A=-(2*mN)/(EN-mN)
    t_GP_01_zz=(-1)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * lat.ZA; t_GP_01_zz_A=-(2*mN)/(EN-mN)

    t_PCAC_G5=(lat.amu/lat.ZP)/mN
    t_PCAC_GP=t_aQ2/(4*mN**2)
    # print(t_PCAC_G5,t_PCAC_GP)

    xUnit=lat.a
    
    pips={
        'GS+_00':[(lat.ZP,'id_j+','P0')],
        # 'GAb_00_zz':[(t_GAzzb_00,'g5gz_j-','Pz')],
        'GA_00_zz':[(t_GAzz_00,'g5gz_j-','Pz')],
        'J5_11':[(lat.ZS,'g5_j-','Pz')],
        # 'GAb_11_tz':[(t_GAtzb_11,'g5gt_j-','Pz')],
        # 'GAb_11_zz':[(t_GAzzb_11,'g5gz_j-','Pz')],
        # 'GAb_11_xx':[(t_GAxxb_11*(1/2),'g5gx_j-','Px'),(t_GAxxb_11*(1/2),'g5gy_j-','Py')],
        'GA_11_tz':[(t_GAtz_11,'g5gt_j-','Pz')],
        'GA_11_zz':[(t_GAzz_11,'g5gz_j-','Pz')],
        'GA_11_xx':[(t_GAxx_11*(1/2),'g5gx_j-','Px'),(t_GAxx_11*(1/2),'g5gy_j-','Py')],
        
        'G5b_01':[(t_G5b_01,'g5_j-','Pz')],
        'GA_01':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
        'GPb_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
        'GPb_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
        
        'PCAC_1':[((-1)*(-(EN-mN)),'g5gt_j-','Pz'),((-1)*(1j*qk),'g5gz_j-','Pz'),((-1)*2*lat.amu,'g5_j-','Pz')],
        'PCAC_1_re':[((-1)*(-(EN-mN))*lat.ZA,'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA,'g5gz_j-','Pz'),((-1)*2*lat.amu/lat.ZP*lat.ZS,'g5_j-','Pz')],
        
        'rPCAC_up_1_Ptz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
        'rPCAC_up_1_Pzz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
    
        'rPCAC_1_Ptz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
        'rPCAC_1_Pzz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
    }[FF]
    
    fig, axs = yu.getFigAxs(2,len(tRefs),sharex=True,sharey=True if ylim1==None else False)
    yu.addRowHeader(axs,['noJL','all'])
    yu.addColHeader(axs,['tRef={}'.format(tRef) for tRef in tRefs])
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    # fig.suptitle(title)
    
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    proj2mat={
        'P0':np.array([[1,0],[0,1]]),
        'Px':np.array([[0,1],[1,0]]),
        'Py':np.array([[0,-1j],[1j,0]]),
        'Pz':np.array([[1,0],[0,-1]]),
    }
    for irow,diags in [(0,yu.diags_jLoopless),(1,yu.diags_all)]:
        for icol,tRef in enumerate(tRefs):
            needsVEV= pta==ptb and 'N-j' in diags and FF in ['GS+_00','J5_11']
            for case in ['noGEVP','GEVP']:
                pta,ofas=ptofas; ptb,ofbs=ptofbs
                if case=='noGEVP':
                    ofas=[ofas[0]]; ofbs=[ofbs[0]]
                    
                Nparas=[yu.getNpar(op) for op in yu.getops(pta,'l1',ofas)]
                Nparbs=[yu.getNpar(op) for op in yu.getops(ptb,'l1',ofbs)]
                maska=np.array([[0 if l==r==2 else 1 for l in Nparas] for r in Nparas])
                maskb=np.array([[0 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
                # maska=np.array([[1 if l==r==2 else 1 for l in Nparas] for r in Nparas])
                # maskb=np.array([[1 if l==r==2 else 1 for l in Nparbs] for r in Nparbs]) 
                
                def t_get(tf,t_insert):
                    t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                        for lb in ['l1','l2']] for la in ['l1','l2']]
                    return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
                
                dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
                dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                        get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
                datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00'] else 'g5_j-']]
                dat=[dat3pt,dat2pt,datVEVj]
                
                AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

                def func(dat):
                    t=yu.meanDeep(dat)
                    t3pt,t2pt,tVEVj=t
                    
                    (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
                    (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
                    (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
                    (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
                    
                    eVec_al1=get_eVec(eVecs_al1)
                    eVec_al2=get_eVec(eVecs_al2)
                    eVec_bl1=get_eVec(eVecs_bl1)
                    eVec_bl2=get_eVec(eVecs_bl2)
                    
                    t2pt_a=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                            np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
                    t2pt_b=(np.einsum('i,tij,j,ij->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1,maskb)+
                            np.einsum('i,tij,j,ij->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2,maskb))/2
                    
                    if needsVEV:
                        proj=proj2mat[pips[0][2]]
                        t2pt_sub=np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)*proj[0,0]+\
                            np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska)*proj[1,1]
                    
                    eVec_a=np.array([eVec_al1,eVec_al2])
                    eVec_b=np.array([eVec_bl1,eVec_bl2])
                    
                    if FF not in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
                        def t_get2(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                res+=prefactor*np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i_pip*len(tfs)+i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                            return res

                        t=[(
                            t_get2(i)
                            -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                        )/np.sqrt(
                            np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                            np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                            np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
                        )
                        for i,tf in enumerate(tfs)]
                    else:
                        def t_get2_N(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                if i_pip<2:
                                    continue
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                res+=prefactor*np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i_pip*len(tfs)+i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                            return res
                        def t_get2_D(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                if i_pip>=2:
                                    continue
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                res+=prefactor*np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i_pip*len(tfs)+i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                            return res

                        t=[(
                            t_get2_N(i)/t_get2_D(i)
                        )
                        for i,tf in enumerate(tfs)]
                        
                
                    t=yu.prefactorDeep(t,1)
                    return t
                
                (mean,err,cov)=yu.jackknife(dat,func)

                for i_tf,tf in enumerate(tfs):
                    tMean=mean[i_tf];tErr=err[i_tf]
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
                    # axs[irow,icol].errorbar(np.arange(1,tf),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
                # axs[irow,icol].set_xticks(nprange(-6,7,1))
                axs[irow,icol].set_ylim(ylim1)
            if ylim1==None:
                axs[0,0].axis('auto')
    
    plt.tight_layout()
    plt.savefig('fig/'+FF+'_'+label+'.pdf')
    plt.close()

In [ ]:
tRefs=[1,2,3,4,5,6,7,8]
# for FF in ['GS+_00','GA_00_zz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),FF,tRefs=tRefs)
# for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
#     runFF(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)
# for FF in ['G5b_01','GA_01','GPb_01_tz','GPb_01_zz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)
# for FF in ['rPCAC_up_1_Ptz','rPCAC_up_1_Pzz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)
# for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)
# for FF in ['PCAC_1','PCAC_1_re']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)

In [ ]:
# tRefs=[1,2,3,4,5,6,7,8]
# for FF in ['GPb_01_tz','GPb_01_zz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs,label='compare',ylim1=[15,35])

In [ ]:
# bare PCAC

tfs=[10,12,14]
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,:,0]/t[:,None]
    # return t[10]
    return np.mean(t[10:14+1,:],axis=0)

def runFF(ptofas,ptofbs,FF,tRefs=[1],label='',ylim1=None):
    
    mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
    t_Fpi=92.9/lat.aInv
    t_aQ2=(qk**2-(EN-mN)**2)
    t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
    t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
    t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P
    
    t_A2pNN_0=mN/t_Fpi
    t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
    t_52pNN_1=(lat.amu/lat.ZP)*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
    t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1
    
    t_GAzz_00=(-1)*(2*mN*(mN+mN))/(2j*((mN+mN)**2)) * lat.ZA; t_GAzzb_00=t_GAzz_00 * t_A2pNN_0
    t_GAtz_11=(-1)*(2*EN*(EN+mN))/(-4*(EN+mN)*qk) * lat.ZA; t_GAtzb_11=t_GAtz_11 * t_A2pNN_0
    t_GAzz_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2+qk**2)) * lat.ZA; t_GAzzb_11=t_GAzz_11 * t_A2pNN_0
    t_GAxx_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2)) * lat.ZA; t_GAxxb_11=t_GAxx_11 * t_A2pNN_0
    
    t_GA_01=(-1)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * 1
    t_G5_01=(-1*np.sqrt(EN*(EN+mN)/2))/qk * 1; t_G5b_01=t_G5_01 * t_52pNN_1
    t_GP_01_tz=(-1)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * 1; t_GP_01_tz_A=-(2*mN)/(EN-mN)
    t_GP_01_zz=(-1)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * 1; t_GP_01_zz_A=-(2*mN)/(EN-mN)

    t_PCAC_G5=(lat.amu)/mN
    t_PCAC_GP=t_aQ2/(4*mN**2)
    # print(t_PCAC_G5,t_PCAC_GP)

    xUnit=lat.a
    
    pips={
        'GS+_00':[(lat.ZP,'id_j+','P0')],
        # 'GAb_00_zz':[(t_GAzzb_00,'g5gz_j-','Pz')],
        'GA_00_zz':[(t_GAzz_00,'g5gz_j-','Pz')],
        'J5_11':[(lat.ZS,'g5_j-','Pz')],
        # 'GAb_11_tz':[(t_GAtzb_11,'g5gt_j-','Pz')],
        # 'GAb_11_zz':[(t_GAzzb_11,'g5gz_j-','Pz')],
        # 'GAb_11_xx':[(t_GAxxb_11*(1/2),'g5gx_j-','Px'),(t_GAxxb_11*(1/2),'g5gy_j-','Py')],
        'GA_11_tz':[(t_GAtz_11,'g5gt_j-','Pz')],
        'GA_11_zz':[(t_GAzz_11,'g5gz_j-','Pz')],
        'GA_11_xx':[(t_GAxx_11*(1/2),'g5gx_j-','Px'),(t_GAxx_11*(1/2),'g5gy_j-','Py')],
        
        'G5b_01':[(t_G5b_01,'g5_j-','Pz')],
        'GA_01':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
        'GPb_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
        'GPb_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
        
        'rPCAC_up_1_Ptz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
        'rPCAC_up_1_Pzz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
    
        'rPCAC_1_Ptz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
        'rPCAC_1_Pzz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
    }[FF]
    
    fig, axs = yu.getFigAxs(2,len(tRefs),sharey=True)
    yu.addRowHeader(axs,['noJL','all'])
    yu.addColHeader(axs,['tRef={}'.format(tRef) for tRef in tRefs])
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    # fig.suptitle(title)
    
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    proj2mat={
        'P0':np.array([[1,0],[0,1]]),
        'Px':np.array([[0,1],[1,0]]),
        'Py':np.array([[0,-1j],[1j,0]]),
        'Pz':np.array([[1,0],[0,-1]]),
    }
    for irow,diags in [(0,yu.diags_jLoopless),(1,yu.diags_all)]:
        for icol,tRef in enumerate(tRefs):
            needsVEV= pta==ptb and 'N-j' in diags and FF in ['GS+_00','J5_11']
            for case in ['noGEVP','GEVP']:
                pta,ofas=ptofas; ptb,ofbs=ptofbs
                if case=='noGEVP':
                    ofas=[ofas[0]]; ofbs=[ofbs[0]]
                    
                Nparas=[yu.getNpar(op) for op in yu.getops(pta,'l1',ofas)]
                Nparbs=[yu.getNpar(op) for op in yu.getops(ptb,'l1',ofbs)]
                maska=np.array([[0 if l==r==2 else 1 for l in Nparas] for r in Nparas])
                maskb=np.array([[0 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
                # maska=np.array([[1 if l==r==2 else 1 for l in Nparas] for r in Nparas])
                # maskb=np.array([[1 if l==r==2 else 1 for l in Nparbs] for r in Nparbs]) 
                
                def t_get(tf,t_insert):
                    t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                        for lb in ['l1','l2']] for la in ['l1','l2']]
                    return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
                
                dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
                dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                        get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
                datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00'] else 'g5_j-']]
                dat=[dat3pt,dat2pt,datVEVj]
                
                AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

                def func(dat):
                    t=yu.meanDeep(dat)
                    t3pt,t2pt,tVEVj=t
                    
                    (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
                    (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
                    (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
                    (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
                    
                    eVec_al1=get_eVec(eVecs_al1)
                    eVec_al2=get_eVec(eVecs_al2)
                    eVec_bl1=get_eVec(eVecs_bl1)
                    eVec_bl2=get_eVec(eVecs_bl2)
                    
                    t2pt_a=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                            np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
                    t2pt_b=(np.einsum('i,tij,j,ij->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1,maskb)+
                            np.einsum('i,tij,j,ij->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2,maskb))/2
                    
                    if needsVEV:
                        proj=proj2mat[pips[0][2]]
                        t2pt_sub=np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)*proj[0,0]+\
                            np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska)*proj[1,1]
                    
                    eVec_a=np.array([eVec_al1,eVec_al2])
                    eVec_b=np.array([eVec_bl1,eVec_bl2])
                    
                    if FF not in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
                        def t_get2(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                res+=prefactor*np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i_pip*len(tfs)+i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                            return res

                        t=[(
                            t_get2(i)
                            -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                        )/np.sqrt(
                            np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                            np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                            np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
                        )
                        for i,tf in enumerate(tfs)]
                    else:
                        def t_get2_N(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                if i_pip<2:
                                    continue
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                res+=prefactor*np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i_pip*len(tfs)+i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                            return res
                        def t_get2_D(i):
                            res=0
                            for i_pip,pip in enumerate(pips):
                                if i_pip>=2:
                                    continue
                                prefactor,insert,proj=pip
                                proj=proj2mat[proj]
                                res+=prefactor*np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i_pip*len(tfs)+i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                            return res

                        t=[(
                            t_get2_N(i)/t_get2_D(i)
                        )
                        for i,tf in enumerate(tfs)]
                        
                
                    t=yu.prefactorDeep(t,1)
                    return t
                
                (mean,err,cov)=yu.jackknife(dat,func)

                for i_tf,tf in enumerate(tfs):
                    tMean=mean[i_tf];tErr=err[i_tf]
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
                    # axs[irow,icol].errorbar(np.arange(1,tf),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
                # axs[irow,icol].set_xticks(nprange(-6,7,1))
                axs[irow,icol].set_ylim(ylim1)
            axs[0,0].axis('auto')
    
    plt.tight_layout()
    plt.savefig('fig/'+FF+'_'+label+'.pdf')
    plt.close()
    
tRefs=[1,2,3,4,5,6,7,8]
# for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs,label='bare')